In [99]:
import pandas as pd
import numpy as np
from glob2 import glob
files = glob('C:\B1\CUST*.csv')

print(files)
df = pd.concat(       #合併客戶輪廓檔和 現金流檔
    (pd.read_csv(file,encoding='big5',usecols=['CUST_RK','EVENT','EVENT_TYPE1','EVENT_DT',],
    dtype={'CUST_RK':int,'EVENT':str,'EVENT_TYPE1':str,'EVENT_DT':str})for file in files),ignore_index=True)



#將整理過的資料用csv格式輸出
#df.to_csv('data_compilation.csv',index=False,encoding='big5')  #將excel資料整理完合併輸出，方便查看完整的data

['C:\\B1\\CUST_CASHFLOW_TRAIN_FIN.csv', 'C:\\B1\\CUST_JOURNEY_TRAIN_FIN.csv']


In [100]:
df

,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT
0,7,預期金流,1.滿期金,19OCT2019
1,7,預期金流,1.滿期金,30NOV2019
2,8,預期金流,3.繳費期滿,04OCT2020
3,25,預期金流,3.繳費期滿,20SEP2019
4,53,預期金流,2.還本金,18MAY2019
...,...,...,...,...
289505,197814,04.電話客服,06.貸/還款,14AUG2020
289506,197814,02.申請契變,A1.要被保人聯絡資訊變更(地址/電話/Email),18AUG2020
289507,197814,02.申請契變,N1.續期保費改為轉帳/信用卡繳費,18AUG2020
289508,197814,02.申請契變,A1.要被保人聯絡資訊變更(地址/電話/Email),20AUG2020


In [101]:
df['EVENT_DT'] = pd.to_datetime(df['EVENT_DT'])    #讓日期比較好識別

In [102]:
print(df['EVENT'].describe())
df

count      289510
unique          6
top       03.數位接觸
freq        80429
Name: EVENT, dtype: object


,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT
0,7,預期金流,1.滿期金,2019-10-19
1,7,預期金流,1.滿期金,2019-11-30
2,8,預期金流,3.繳費期滿,2020-10-04
3,25,預期金流,3.繳費期滿,2019-09-20
4,53,預期金流,2.還本金,2019-05-18
...,...,...,...,...
289505,197814,04.電話客服,06.貸/還款,2020-08-14
289506,197814,02.申請契變,A1.要被保人聯絡資訊變更(地址/電話/Email),2020-08-18
289507,197814,02.申請契變,N1.續期保費改為轉帳/信用卡繳費,2020-08-18
289508,197814,02.申請契變,A1.要被保人聯絡資訊變更(地址/電話/Email),2020-08-20


In [103]:
mask = df['EVENT'] == '01.申購保單'     #刪除再購的資料
loc = df.loc[mask]                     
df = df.drop(loc.index,axis = 0)    
print(df['EVENT'].describe())
df

count      272502
unique          5
top       03.數位接觸
freq        80429
Name: EVENT, dtype: object


,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT
0,7,預期金流,1.滿期金,2019-10-19
1,7,預期金流,1.滿期金,2019-11-30
2,8,預期金流,3.繳費期滿,2020-10-04
3,25,預期金流,3.繳費期滿,2019-09-20
4,53,預期金流,2.還本金,2019-05-18
...,...,...,...,...
289504,197814,04.電話客服,06.貸/還款,2019-08-21
289505,197814,04.電話客服,06.貸/還款,2020-08-14
289506,197814,02.申請契變,A1.要被保人聯絡資訊變更(地址/電話/Email),2020-08-18
289507,197814,02.申請契變,N1.續期保費改為轉帳/信用卡繳費,2020-08-18


In [104]:
from sklearn.preprocessing import LabelEncoder    #建模前處理，將字串轉成數值進行編碼
labelencoder = LabelEncoder()         
df['EVENT'] = labelencoder.fit_transform(df['EVENT'])
df['EVENT_TYPE1'] = labelencoder.fit_transform(df['EVENT_TYPE1'])

In [105]:
df

,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT
0,7,4,0,2019-10-19
1,7,4,0,2019-11-30
2,8,4,2,2020-10-04
3,25,4,2,2019-09-20
4,53,4,1,2019-05-18
...,...,...,...,...
289504,197814,2,14,2019-08-21
289505,197814,2,14,2020-08-14
289506,197814,0,35,2020-08-18
289507,197814,0,74,2020-08-18


In [106]:
yu = pd.read_csv(r'C:\A1\journey_df_train.csv',encoding='big5')    #匯入再購率時間的資料

In [107]:
verify = yu[['MONTH_SINCE_LAST_BUY']]

In [108]:
verify

,MONTH_SINCE_LAST_BUY
0,-1
1,-1
2,-1
3,-1
4,-1
...,...
272497,3
272498,3
272499,-1
272500,-1


In [109]:
df.insert(4,column = 'verify',value = verify)   #新增一個欄位，將再購率資料欄位併入df資料中

In [110]:
df

,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT,verify
0,7,4,0,2019-10-19,-1.0
1,7,4,0,2019-11-30,-1.0
2,8,4,2,2020-10-04,-1.0
3,25,4,2,2019-09-20,-1.0
4,53,4,1,2019-05-18,-1.0
...,...,...,...,...,...
289504,197814,2,14,2019-08-21,NaN
289505,197814,2,14,2020-08-14,NaN
289506,197814,0,35,2020-08-18,NaN
289507,197814,0,74,2020-08-18,NaN


In [111]:
print(type(df['verify'].mode()))    #想用眾數填補NA，但無法填入，因為資料型態為series

<class 'pandas.core.series.Series'>


In [112]:
df['verify'] = df['verify'].fillna(df['verify'].mode()[0])    #將verify欄位用眾數去填補

In [113]:
df

,CUST_RK,EVENT,EVENT_TYPE1,EVENT_DT,verify
0,7,4,0,2019-10-19,-1.0
1,7,4,0,2019-11-30,-1.0
2,8,4,2,2020-10-04,-1.0
3,25,4,2,2019-09-20,-1.0
4,53,4,1,2019-05-18,-1.0
...,...,...,...,...,...
289504,197814,2,14,2019-08-21,-1.0
289505,197814,2,14,2020-08-14,-1.0
289506,197814,0,35,2020-08-18,-1.0
289507,197814,0,74,2020-08-18,-1.0


In [114]:
#先將feature做分類
num_feature = []   #區分為數值型特徵
for dtype,feature in zip(df.dtypes, df.columns):  #類似00,01去指
    if dtype == 'float64' or dtype == 'int64': #由dtype去判斷型態
        num_feature.append(feature)    #符合再將feature插入空陣列中
print(f'{len(num_feature)} Numeric Features : {num_feature}\n')

1 Numeric Features : ['verify']



In [115]:
category_feature = [] #區分為類別型資料
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        category_feature.append(feature)
print(f'{len(category_feature)} category Features : {category_feature}\n')

0 category Features : []



In [116]:
import featuretools as ft     #利用特徵工程，找出更多相關的特徵
print('Feature tool version {}'.format(ft.__version__))

#創一個實體集(Entity set)
es = ft.EntitySet(id = 'repurchase_rate')    #以找出再購相關的特徵為目標

Feature tool version 0.27.1


In [117]:
es = es.entity_from_dataframe(entity_id = 'repurchase_rate',   #唯一可和entity連結的ID
                              dataframe = df,                  #可和資料連結的框架
                             make_index=True,
                              index = 'id',
                              time_index = 'EVENT_DT')   #index不唯一，導入make_index = True

In [118]:
es

Entityset: repurchase_rate
  Entities:
    repurchase_rate [Rows: 272502, Columns: 6]
  Relationships:
    No relationships

In [119]:
es['repurchase_rate']    #看entity set 內容

Entity: repurchase_rate
  Variables:
    id (dtype: index)
    CUST_RK (dtype: numeric)
    EVENT (dtype: numeric)
    EVENT_TYPE1 (dtype: numeric)
    EVENT_DT (dtype: datetime_time_index)
    verify (dtype: numeric)
  Shape:
    (Rows: 272502, Columns: 6)

In [120]:
es.normalize_entity(base_entity_id='repurchase_rate',
                    new_entity_id='items',              #從repurchase_rate中再建立新表
                    index = 'verify')

Entityset: repurchase_rate
  Entities:
    repurchase_rate [Rows: 272502, Columns: 6]
    items [Rows: 5, Columns: 2]
  Relationships:
    repurchase_rate.verify -> items.verify

In [121]:
feature_matrix,feature_names = ft.dfs(entityset=es,   #使用DFS來自動建立新特徵
                                     target_entity = 'repurchase_rate',
                                     max_depth = 2,
                                     verbose  = 1,
                                     n_jobs = 3)

Built 31 features
EntitySet scattered to 3 workers in 13 seconds                                                                         
Elapsed: 00:05 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████


In [122]:
feature_matrix

,CUST_RK,EVENT,EVENT_TYPE1,verify,DAY(EVENT_DT),MONTH(EVENT_DT),WEEKDAY(EVENT_DT),YEAR(EVENT_DT),items.COUNT(repurchase_rate),items.MAX(repurchase_rate.CUST_RK),...,items.STD(repurchase_rate.CUST_RK),items.STD(repurchase_rate.EVENT),items.STD(repurchase_rate.EVENT_TYPE1),items.SUM(repurchase_rate.CUST_RK),items.SUM(repurchase_rate.EVENT),items.SUM(repurchase_rate.EVENT_TYPE1),items.DAY(first_repurchase_rate_time),items.MONTH(first_repurchase_rate_time),items.WEEKDAY(first_repurchase_rate_time),items.YEAR(first_repurchase_rate_time)
id,,,,,,,,,,,,,,,,,,,,,
4228,10011,4,1,-1.0,1,1,1,2019,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
5485,13668,4,1,-1.0,1,1,1,2019,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
5935,14487,4,1,-1.0,1,1,1,2019,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
9858,21943,4,1,-1.0,1,1,1,2019,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
10326,22535,4,1,-1.0,1,1,1,2019,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269193,194691,1,70,-1.0,31,12,3,2020,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
269900,195211,1,70,-1.0,31,12,3,2020,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019
271047,196626,2,5,-1.0,31,12,3,2020,240398,197814,...,57681.156495,1.562739,29.883392,2.196556e+10,507836,7895872,1,1,1,2019


In [123]:
feature_names   #所有特徵名稱

[<Feature: CUST_RK>,
 <Feature: EVENT>,
 <Feature: EVENT_TYPE1>,
 <Feature: verify>,
 <Feature: DAY(EVENT_DT)>,
 <Feature: MONTH(EVENT_DT)>,
 <Feature: WEEKDAY(EVENT_DT)>,
 <Feature: YEAR(EVENT_DT)>,
 <Feature: items.COUNT(repurchase_rate)>,
 <Feature: items.MAX(repurchase_rate.CUST_RK)>,
 <Feature: items.MAX(repurchase_rate.EVENT)>,
 <Feature: items.MAX(repurchase_rate.EVENT_TYPE1)>,
 <Feature: items.MEAN(repurchase_rate.CUST_RK)>,
 <Feature: items.MEAN(repurchase_rate.EVENT)>,
 <Feature: items.MEAN(repurchase_rate.EVENT_TYPE1)>,
 <Feature: items.MIN(repurchase_rate.CUST_RK)>,
 <Feature: items.MIN(repurchase_rate.EVENT)>,
 <Feature: items.MIN(repurchase_rate.EVENT_TYPE1)>,
 <Feature: items.SKEW(repurchase_rate.CUST_RK)>,
 <Feature: items.SKEW(repurchase_rate.EVENT)>,
 <Feature: items.SKEW(repurchase_rate.EVENT_TYPE1)>,
 <Feature: items.STD(repurchase_rate.CUST_RK)>,
 <Feature: items.STD(repurchase_rate.EVENT)>,
 <Feature: items.STD(repurchase_rate.EVENT_TYPE1)>,
 <Feature: items.SUM(r

In [124]:
p = pd.read_csv(r'C:\A1\x_df.csv',encoding='big5')    #從profile補新的特徵進去

In [125]:
p

,CUST_RK,AGE,INSURED_SEX,WEALTH_LEVEL_REV,loytal_level_AG,CLIENT_INCOME,total_aum,TOPCARD,DIGI_FLG,ternure_m_AG,...,B7.保L,A7.FL,02.收L,A1.要L,CLUBL,L.保單L,2.還20,3.繳20,2.還F,3.繳F
0,2,47,M,W5,L5,748000,174600,0,0,273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,13,M,W1,L5,377000,230580,1,0,156,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,22,M,W5,L5,377000,64800,0,0,235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,31,M,W1,L5,640000,2894940,0,0,242,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,22,M,W1,L5,377000,8295039,0,0,244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,197811,52,F,W3,L5,540000,8765,0,0,143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99993,197812,51,F,W1,L3,1260000,5093285,1,1,229,...,9.0,9.0,0.0,0.0,0.0,0.0,6.0,3.0,12.0,12.0
99994,197813,51,F,W4,L5,450000,452430,0,0,314,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99995,197814,50,F,W1,L3,13500000,167578,1,0,232,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0


In [126]:
new = pd.concat([feature_matrix,p])

In [127]:
new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372499 entries, 4228 to 99996
Columns: 112 entries, CUST_RK to 3.繳F
dtypes: float64(108), int64(1), object(3)
memory usage: 321.1+ MB


In [128]:
new['INSURED_SEX'] = labelencoder.fit_transform(new['INSURED_SEX'])              #將一些object利用labelencoding轉換
new['WEALTH_LEVEL_REV'] = labelencoder.fit_transform(new['WEALTH_LEVEL_REV'])
new['loytal_level_AG'] = labelencoder.fit_transform(new['loytal_level_AG'])

In [129]:
new

,CUST_RK,EVENT,EVENT_TYPE1,verify,DAY(EVENT_DT),MONTH(EVENT_DT),WEEKDAY(EVENT_DT),YEAR(EVENT_DT),items.COUNT(repurchase_rate),items.MAX(repurchase_rate.CUST_RK),...,B7.保L,A7.FL,02.收L,A1.要L,CLUBL,L.保單L,2.還20,3.繳20,2.還F,3.繳F
4228,10011,4.0,1.0,-1.0,1.0,1.0,1.0,2019.0,240398.0,197814.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5485,13668,4.0,1.0,-1.0,1.0,1.0,1.0,2019.0,240398.0,197814.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5935,14487,4.0,1.0,-1.0,1.0,1.0,1.0,2019.0,240398.0,197814.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9858,21943,4.0,1.0,-1.0,1.0,1.0,1.0,2019.0,240398.0,197814.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10326,22535,4.0,1.0,-1.0,1.0,1.0,1.0,2019.0,240398.0,197814.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,197811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99993,197812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,9.0,0.0,0.0,0.0,0.0,6.0,3.0,12.0,12.0
99994,197813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99995,197814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0


In [132]:
#new['EVENT_TYPE1']=new['EVENT_TYPE1'].fillna((df['EVENT_TYPE1'].mode())

new['EVENT']=new['EVENT'].fillna(new['EVENT'].mode()[0])
new['EVENT_TYPE1']=new['EVENT_TYPE1'].fillna(new['EVENT_TYPE1'].mode()[0])
new['verify'] = new['verify'].fillna(new['verify'].mode()[0])    
new['DAY(EVENT_DT)']=new['DAY(EVENT_DT)'].fillna(value=new['DAY(EVENT_DT)'].mean())
new['MONTH(EVENT_DT)']=new['MONTH(EVENT_DT)'].fillna(value=new['MONTH(EVENT_DT)'].mean())
new['WEEKDAY(EVENT_DT)']=new['WEEKDAY(EVENT_DT)'].fillna(value=new['WEEKDAY(EVENT_DT)'].mean())
new['YEAR(EVENT_DT)']=new['YEAR(EVENT_DT)'].fillna(value=new['YEAR(EVENT_DT)'].mean())
new['items.COUNT(repurchase_rate)']=new['items.COUNT(repurchase_rate)'].fillna(value=new['items.COUNT(repurchase_rate)'].mean())
new['items.MAX(repurchase_rate.CUST_RK)']=new['items.MAX(repurchase_rate.CUST_RK)'].fillna(value=new['items.MAX(repurchase_rate.CUST_RK)'].mean())
new['items.MAX(repurchase_rate.EVENT)']=new['items.MAX(repurchase_rate.EVENT)'].fillna(value=new['items.MAX(repurchase_rate.EVENT)'].mean())
new['items.MAX(repurchase_rate.EVENT_TYPE1)']=new['items.MAX(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.MAX(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.MEAN(repurchase_rate.CUST_RK)']=new['items.MEAN(repurchase_rate.CUST_RK)'].fillna(value=new['items.MEAN(repurchase_rate.CUST_RK)'].mean())
new['items.MEAN(repurchase_rate.EVENT)']=new['items.MEAN(repurchase_rate.EVENT)'].fillna(value=new['items.MEAN(repurchase_rate.EVENT)'].mean())
new['items.MEAN(repurchase_rate.EVENT_TYPE1)']=new['items.MEAN(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.MEAN(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.MIN(repurchase_rate.CUST_RK)']=new['items.MIN(repurchase_rate.CUST_RK)'].fillna(value=new['items.MIN(repurchase_rate.CUST_RK)'].mean())
new['items.MIN(repurchase_rate.EVENT)']=new['items.MIN(repurchase_rate.EVENT)'].fillna(value=new['items.MIN(repurchase_rate.EVENT)'].mean())
new['items.MIN(repurchase_rate.EVENT_TYPE1)']=new['items.MIN(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.MIN(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.SKEW(repurchase_rate.CUST_RK)']=new['items.SKEW(repurchase_rate.CUST_RK)'].fillna(value=new['items.SKEW(repurchase_rate.CUST_RK)'].mean())
new['items.SKEW(repurchase_rate.EVENT)']=new['items.SKEW(repurchase_rate.EVENT)'].fillna(value=new['items.SKEW(repurchase_rate.EVENT)'].mean())
new['items.SKEW(repurchase_rate.EVENT_TYPE1)']=new['items.SKEW(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.SKEW(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.STD(repurchase_rate.CUST_RK)']=new['items.STD(repurchase_rate.CUST_RK)'].fillna(value=new['items.STD(repurchase_rate.CUST_RK)'].mean())
new['items.STD(repurchase_rate.EVENT)']=new['items.STD(repurchase_rate.EVENT)'].fillna(value=new['items.STD(repurchase_rate.EVENT)'].mean())
new['items.STD(repurchase_rate.EVENT_TYPE1)']=new['items.STD(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.STD(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.SUM(repurchase_rate.CUST_RK)']=new['items.SUM(repurchase_rate.CUST_RK)'].fillna(value=new['items.SUM(repurchase_rate.CUST_RK)'].mean())
new['items.SUM(repurchase_rate.EVENT_TYPE1)']=new['items.SUM(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.SUM(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.SUM(repurchase_rate.EVENT_TYPE1)']=new['items.SUM(repurchase_rate.EVENT_TYPE1)'].fillna(value=new['items.SUM(repurchase_rate.EVENT_TYPE1)'].mean())
new['items.DAY(first_repurchase_rate_time)']=new['items.DAY(first_repurchase_rate_time)'].fillna(value=new['items.DAY(first_repurchase_rate_time)'].mean())
new['items.MONTH(first_repurchase_rate_time)']=new['items.MONTH(first_repurchase_rate_time)'].fillna(value=new['items.MONTH(first_repurchase_rate_time)'].mean())
new['items.WEEKDAY(first_repurchase_rate_time)']=new['items.WEEKDAY(first_repurchase_rate_time)'].fillna(value=new['items.WEEKDAY(first_repurchase_rate_time)'].mean())
new['items.YEAR(first_repurchase_rate_time)']=new['items.YEAR(first_repurchase_rate_time)'].fillna(value=new['items.YEAR(first_repurchase_rate_time)'].mean())
new['AGE']=new['AGE'].fillna(value=new['AGE'].mean())
new['INSURED_SEX']=new['INSURED_SEX'].fillna(value=new['INSURED_SEX'].mean())
new['WEALTH_LEVEL_REV']=new['WEALTH_LEVEL_REV'].fillna(value=new['WEALTH_LEVEL_REV'].mean())
new['loytal_level_AG']=new['loytal_level_AG'].fillna(value=new['loytal_level_AG'].mean())
new['CLIENT_INCOME']=new['CLIENT_INCOME'].fillna(value=new['CLIENT_INCOME'].mean())
new['total_aum']=new['total_aum'].fillna(value=new['total_aum'].mean())
new['TOPCARD']=new['TOPCARD'].fillna(value=new['TOPCARD'].mean())
new['DIGI_FLG']=new['DIGI_FLG'].fillna(value=new['DIGI_FLG'].mean())
new['ternure_m_AG']=new['ternure_m_AG'].fillna(value=new['ternure_m_AG'].mean())
new['recency_m_AG']=new['recency_m_AG'].fillna(value=new['recency_m_AG'].mean())
new['product_density']=new['product_density'].fillna(value=new['product_density'].mean())
new['tot_pol_cnt_his']=new['tot_pol_cnt_his'].fillna(value=new['tot_pol_cnt_his'].mean())
new['tot_txn_cnt_his_AG']=new['tot_txn_cnt_his_AG'].fillna(value=new['tot_txn_cnt_his_AG'].mean())
new['TOT_AFYP']=new['TOT_AFYP'].fillna(value=new['TOT_AFYP'].mean())

In [133]:
new

,CUST_RK,EVENT,EVENT_TYPE1,verify,DAY(EVENT_DT),MONTH(EVENT_DT),WEEKDAY(EVENT_DT),YEAR(EVENT_DT),items.COUNT(repurchase_rate),items.MAX(repurchase_rate.CUST_RK),...,B7.保L,A7.FL,02.收L,A1.要L,CLUBL,L.保單L,2.還20,3.繳20,2.還F,3.繳F
4228,10011,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5485,13668,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5935,14487,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9858,21943,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10326,22535,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,197811,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99993,197812,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,9.0,9.0,0.0,0.0,0.0,0.0,6.0,3.0,12.0,12.0
99994,197813,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99995,197814,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0


In [134]:
new = new.fillna(0) 

In [135]:
new.columns

Index(['CUST_RK', 'EVENT', 'EVENT_TYPE1', 'verify', 'DAY(EVENT_DT)',
       'MONTH(EVENT_DT)', 'WEEKDAY(EVENT_DT)', 'YEAR(EVENT_DT)',
       'items.COUNT(repurchase_rate)', 'items.MAX(repurchase_rate.CUST_RK)',
       ...
       'B7.保L', 'A7.FL', '02.收L', 'A1.要L', 'CLUBL', 'L.保單L', '2.還20', '3.繳20',
       '2.還F', '3.繳F'],
      dtype='object', length=112)

In [136]:
new

,CUST_RK,EVENT,EVENT_TYPE1,verify,DAY(EVENT_DT),MONTH(EVENT_DT),WEEKDAY(EVENT_DT),YEAR(EVENT_DT),items.COUNT(repurchase_rate),items.MAX(repurchase_rate.CUST_RK),...,B7.保L,A7.FL,02.收L,A1.要L,CLUBL,L.保單L,2.還20,3.繳20,2.還F,3.繳F
4228,10011,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5485,13668,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5935,14487,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9858,21943,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10326,22535,4.0,1.0,-1.0,1.00000,1.000000,1.000000,2019.000000,240398.000000,197814.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99992,197811,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99993,197812,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,9.0,9.0,0.0,0.0,0.0,0.0,6.0,3.0,12.0,12.0
99994,197813,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99995,197814,1.0,70.0,-1.0,16.53142,6.745198,2.461182,2019.522459,213082.764024,196021.855216,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0


In [137]:
new.to_csv(r'C:\A1\renew_feature_0521.csv',encoding='big5')    

In [94]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [95]:
x_new=SelectKBest(chi2,20).fit_transform(feature_matrix,final)     #

C:\Users\linnn\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass k=20 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


ValueError: Input X must be non-negative.

In [1]:
!pip install catboost

     -------------------------------------- 47.0/47.0 kB 592.9 kB/s eta 0:00:00
     ---------------------------------------- 15.2/15.2 MB 2.2 MB/s eta 0:00:00


In [56]:
#開始驗證feature的效果
from catboost import CatBoostRegressor

In [59]:
categorical_features = np.where(feature_matrix.dtypes =='object')[0] 
for i in categorical_features: 
    feature_matrix.iloc[:,i]=feature_matrix.iloc[:,i].astype('str')

In [68]:
feature_matrix.drop(['id'], axis=1, inplace=True) 
train = feature_matrix[:8523] 
test = feature_matrix[8523:]
# removing uneccesary variables 
train.drop(['CUST_RK'], axis=1, inplace=True) test.drop(['CUST_RK'], axis=1, inplace=True)
# identifying categorical features categorical_features = np.where(train.dtypes == 'object')[0]

SyntaxError: invalid syntax (<ipython-input-68-410176c7ba54>, line 5)